<a href="https://colab.research.google.com/github/Azimoj/Home-Credit-Default-Risk/blob/main/Credit__scoring_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The data contains several number of relational tables. We'll process each one of them separately, and then finally in the end, merge all of them together.

## **Preprocessing Tables**

In [7]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [8]:
# Data Structures
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib

import warnings
warnings.filterwarnings('ignore')
from datetime import datetime

import time
import gc

from sklearn.preprocessing import LabelEncoder
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
import xgboost as xgb
import lightgbm as lgb


In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [38]:
#### Functions of the project.
def grab_col_names(dataframe, cat_th=10, car_th=20):
    """
        It gives the names of categorical, numerical and categorical but cardinal variables in the data set.
        Note: Categorical variables with numerical appearance are also included in categorical variables.
    Parameters
    ------
        dataframe: dataframe
               The dataframe from which variable names are to be retrieved
         cat_th: int, optional
                 class threshold value for numeric but categorical variables
         car_th: int, optinal
                 class threshold for categorical but cardinal variables
    Returns
    ------
          The dataframe from which variable names are to be retrieved
          cat_th: int, optional
                  class threshold value for numeric but categorical variables
          car_th: int, optinal
                  class threshold for categorical but cardinal variables
    Examples
    ------
     import seaborn as sns
         df = sns.load_dataset("iris")
         print(grab_col_names(df))
     notes
     ------
         cat_cols + num_cols + cat_but_car = total number of variables
         num_but_cat is inside cat_cols.
         The sum of the 3 returned lists equals the total number of variables: cat_cols + num_cols + cat_but_car = number of variables
    """

    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]

    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]     

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')

    return cat_cols, num_cols

In [39]:
def high_correlated_cols(dataframe, plot=False, corr_th=0.90):
    corr = dataframe.corr()
    cor_matrix = corr.abs()
    upper_triangle_matrix = cor_matrix.where(np.triu(np.ones(cor_matrix.shape), k=1).astype(np.bool))
    drop_list = [col for col in upper_triangle_matrix.columns if any(upper_triangle_matrix[col] > corr_th)]
    if plot:
        import seaborn as sns
        import matplotlib.pyplot as plt
        sns.set(rc={'figure.figsize': (15, 15)})
        sns.heatmap(corr, cmap="RdBu")
        plt.show()
    return drop_list  

In [40]:
# Defining Sin-cos transformation for cyclic features
def encode(df, col, max_val):
    df[col + '_SIN'] = np.sin(2 * np.pi * df[col] / max_val)
    df[col + '_COS'] = np.cos(2 * np.pi * df[col] / max_val)
    return df

In [41]:
# Defining one-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category=True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na = nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

## **application_train and application_test**

In [31]:
#Download databases
app_test= pd.read_csv('/content/drive/MyDrive/p7/data/application_test.csv', encoding='utf-8', low_memory=False)
app_train= pd.read_csv('/content/drive/MyDrive/p7/data/application_train.csv',low_memory=False)

In [ ]:
app_train.shape

(307511, 122)

In [ ]:
app_test.shape

(48744, 121)

In [32]:
df = app_train.append(app_test).reset_index()

### **Data cleaning**

In [ ]:
df.head()

,index,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,...,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,0,100002,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,...,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,100003,0.0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,...,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,100004,0.0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-815.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,100006,0.0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,0.008019,-19005,-3039,-9833.0,-2437,NaN,1,1,0,1,0,0,Laborers,2.0,2,2,WEDNESDAY,17,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2.0,0.0,-617.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,100007,0.0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.028663,-19932,-3038,-4311.0,-3458,NaN,1,1,0,1,0,0,Core staff,1.0,2,2,THURSDAY,11,0,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-1106.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
# Removing 4 applications with XNA CODE_GENDER (train set)
df = df[df['CODE_GENDER'] != 'XNA']

In [34]:
cat_cols, num_cols = grab_col_names(df, cat_th=10, car_th=20)

Observations: 356251
Variables: 123
cat_cols: 16
num_cols: 107


In [35]:
df.duplicated('SK_ID_CURR').sum()

0

In [40]:
high_list = high_correlated_cols(df[num_cols], plot=False, corr_th=0.98)

In [ ]:
high_list

['AMT_GOODS_PRICE',
 'FLAG_EMP_PHONE',
 'YEARS_BUILD_MODE',
 'FLOORSMAX_MODE',
 'FLOORSMIN_MODE',
 'APARTMENTS_MEDI',
 'BASEMENTAREA_MEDI',
 'YEARS_BEGINEXPLUATATION_MEDI',
 'YEARS_BUILD_MEDI',
 'COMMONAREA_MEDI',
 'ELEVATORS_MEDI',
 'ENTRANCES_MEDI',
 'FLOORSMAX_MEDI',
 'FLOORSMIN_MEDI',
 'LANDAREA_MEDI',
 'LIVINGAPARTMENTS_MEDI',
 'LIVINGAREA_MEDI',
 'NONLIVINGAPARTMENTS_MEDI',
 'NONLIVINGAREA_MEDI',
 'OBS_60_CNT_SOCIAL_CIRCLE']

In [41]:
df.drop(high_list, inplace= True, axis=1)

In [42]:
#I excract the high corelated columns from the numerical columns. 
num_cols = [col for col in num_cols if col not in high_list]

In [43]:
df["NAME_FAMILY_STATUS"].unique()

array(['Single / not married', 'Married', 'Civil marriage', 'Widow',
       'Separated', 'Unknown'], dtype=object)

In [44]:
# this unit has no information so I have deleted
df = df.loc[df["NAME_FAMILY_STATUS"] != "Unknown"]

In [ ]:
"""# this units have no information so I have deleted them.
drop_list = ["FLAG_MOBIL", "FLAG_DOCUMENT_4", "FLAG_DOCUMENT_5", "FLAG_DOCUMENT_7", "FLAG_DOCUMENT_10", "FLAG_DOCUMENT_12",
              "FLAG_DOCUMENT_16", "FLAG_DOCUMENT_17", "FLAG_DOCUMENT_19", "FLAG_DOCUMENT_20", "FLAG_DOCUMENT_21" ]
for col in drop_list:
    cat_cols.pop(drop_list.index(col))
    df.drop(col, inplace= True, axis=1)"""

In [45]:
# NaN values for DAYS_EMPLOYED: 365243 -> nan
df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)

# Changing rare categories of NAME_INCOME_TYPE with the similar categories
df.loc[df['NAME_INCOME_TYPE'] == 'Student', 'NAME_INCOME_TYPE'] = 'State servant'
df.loc[df['NAME_INCOME_TYPE'] == 'Maternity leave', 'NAME_INCOME_TYPE'] = 'Pensioner'
df.loc[df['NAME_INCOME_TYPE'] == 'Unemployed', 'NAME_INCOME_TYPE'] = 'Pensioner'
df.loc[df['NAME_INCOME_TYPE'] == 'Businessman', 'NAME_INCOME_TYPE'] = 'Commercial associate'

In [46]:
# Rare Encoding NAME_HOUSING_TYPE with 'Other'
df.loc[(df['NAME_HOUSING_TYPE'] == 'Office apartment') &
        (df['NAME_HOUSING_TYPE'] == 'Co-op apartment'), 'NAME_HOUSING_TYPE'] = 'Other'

#  Changing unknown family status with the most observed category
df['NAME_FAMILY_STATUS'].replace('Unknown', 'Married', inplace=True)

#  Changing HOUSETYPE_MODE not null values with
df.loc[df['HOUSETYPE_MODE'].notnull(), 'HOUSETYPE_MODE'] = 'house_type_reported'

# Changing weekdays with integer values
weekday_dict = {'MONDAY': 1, 'TUESDAY': 2, 'WEDNESDAY': 3, 'THURSDAY': 4, 'FRIDAY': 5, 'SATURDAY': 6, 'SUNDAY': 7}
df.replace({'WEEKDAY_APPR_PROCESS_START': weekday_dict}, inplace=True)

In [47]:

# Creating sin-cos transformed features
df = encode(df, 'WEEKDAY_APPR_PROCESS_START', 7)
df = encode(df, 'HOUR_APPR_PROCESS_START', 23)
# Deleting initial WEEKDAY_APPR_PROCESS_START and HOUR_APPR_PROCESS_START features
df.drop(['WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START'], axis=1, inplace=True)

# new feature revolving loans
df['NEW_REVOLVING_LOANS']=1 
df.loc[(df['NAME_CONTRACT_TYPE']!='Revolving loans'),'NEW_REVOLVING_LOANS']=0
df['NEW_REVOLVING_LOANS'] = df['NEW_REVOLVING_LOANS'].astype(object)

# new feature has car
df['NEW_HAS_CAR']=1 
df.loc[(df['FLAG_OWN_CAR']!='Y'),'NEW_HAS_CAR']=0
df['NEW_HAS_CAR'] = df['NEW_HAS_CAR'].astype(object)

# new feature has rent
df['NEW_RENT_EXPENSE']=1 
df.loc[(df['FLAG_OWN_REALTY']!='Y'),'NEW_RENT_EXPENSE']=0
df['NEW_RENT_EXPENSE'] = df['NEW_RENT_EXPENSE'].astype(object)

# new feature has rent
df['NEW_HAS_COMPANY']=1 
df.loc[(df['NAME_INCOME_TYPE']!='Commercial associate'),'NEW_HAS_COMPANY']=0
df['NEW_HAS_COMPANY'] = df['NEW_HAS_COMPANY'].astype(object)

df['NEW_LABORERS']=0 
df.loc[(df["OCCUPATION_TYPE"] == "Cleaning staff") | (df["OCCUPATION_TYPE"] == "Cooking staff") | 
        (df["OCCUPATION_TYPE"] == "Drivers") | (df["OCCUPATION_TYPE"] == "Laborers")
      | (df["OCCUPATION_TYPE"] == "Low-skill Laborers") | (df["OCCUPATION_TYPE"] == "Sales staff")
      | (df["OCCUPATION_TYPE"] == "Security staff") | (df["OCCUPATION_TYPE"] == "Waiters/barmen staff"), 'NEW_LABORERS' ] = 1
df['NEW_LABORERS'] = df['NEW_LABORERS'].astype(object)

# New features (percentages)
df['NEW_DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
df['NEW_INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
df['NEW_INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
df['NEW_ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
df['NEW_ANNUITY_CREDIT_RATIO'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']

# Mean of External Sources
df["NEW_EXT_MEAN"] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)

# Product of External Sources
df['NEW_EXT_PROD'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']

# Ages of customers
df['NEW_AGE'] = df['DAYS_BIRTH'] / -365

# NEW_AGE_SEGMENT segments
df.loc[df['NEW_AGE'] <= 34, 'NEW_AGE_SEGMENT'] = 'YOUNG'
df.loc[(df['NEW_AGE'] > 34) & (df['NEW_AGE'] <= 54), 'NEW_AGE_SEGMENT'] = 'MİDDLE'
df.loc[df['NEW_AGE'] > 54, 'NEW_AGE_SEGMENT'] = 'OLD'

# NEW_ACCOMPANIED feature
df.loc[df['NAME_TYPE_SUITE'] == 'Unaccompanied', 'NEW_ACCOMPANIED'] = 0
df.loc[df['NAME_TYPE_SUITE'] != 'Unaccompanied', 'NEW_ACCOMPANIED'] = 1
df.loc[df['NAME_TYPE_SUITE'].isnull(), 'NEW_ACCOMPANIED'] = np.nan

# Social circle with both 30 and 60 days default (binary)
df.loc[(df['DEF_30_CNT_SOCIAL_CIRCLE'] > 0) & (df['DEF_60_CNT_SOCIAL_CIRCLE'] > 0),
        'NEW_DEF_30&60_SOCIAL_CIRCLE'] = 1
df.loc[(df['DEF_30_CNT_SOCIAL_CIRCLE'] == 0) & (df['DEF_60_CNT_SOCIAL_CIRCLE'] == 0),
        'NEW_DEF_30&60_SOCIAL_CIRCLE'] = 0



In [48]:
 # Label encoding
for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
    df[bin_feature], uniques = pd.factorize(df[bin_feature])

In [49]:
# Categorical features with One-Hot encode
df, cat_cols = one_hot_encoder(df, nan_as_category=True)

In [50]:
# Dropping feature named index
df.drop('index', axis=1, inplace=True)

In [ ]:
del app_train,app_test

In [51]:
# blocking garbage collection of all generations (garbage cycle)
# Running a garbage collection process cleans up a huge amount of objects—there are 577 objects in the first generation and three more in the older generations.
# https://stackify.com/python-garbage-collection/

gc.collect()

521

**Backup CSVs**

In [52]:
df.to_csv("/content/drive/MyDrive/p7/data/df_train_final.csv", index=False)


## **bureau_balance.csv and bureau.csv**

**bureau_balance Preprocessing**

This table consists of Monthly balance of each credit for each of the previous credit that the client had with financial institutions other than Home Credit

In [53]:
bb= pd.read_csv('/content/drive/MyDrive/p7/data/bureau_balance.csv',sep=',' , encoding='utf-8', low_memory=False)

In [54]:
# did it due to day or not?'
liste = ['1', '2', '3', '4', '5']
bb['NEW_FLAG'] = bb['STATUS'].apply(lambda x: 1 if (x in liste) else ("X" if x == "X" else 0))

bb, bb_cat = one_hot_encoder(bb)
bb.drop("NEW_FLAG_X", inplace=True, axis=1)
bb_cat.remove('NEW_FLAG_X')

bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
for col in bb_cat:
    bb_aggregations[col] = ['mean']

bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations).reset_index()
bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])

In [55]:
del bb

In [56]:
gc.collect()

204

In [57]:
bb_agg.shape

(817395, 16)

In [58]:
bb_cat

['STATUS_0',
 'STATUS_1',
 'STATUS_2',
 'STATUS_3',
 'STATUS_4',
 'STATUS_5',
 'STATUS_C',
 'STATUS_X',
 'STATUS_nan',
 'NEW_FLAG_0',
 'NEW_FLAG_1',
 'NEW_FLAG_nan']

In [59]:
bb_agg.isnull().sum().sort_values(ascending=False)

SK_ID_BUREAU_          0
MONTHS_BALANCE_MIN     0
MONTHS_BALANCE_MAX     0
MONTHS_BALANCE_SIZE    0
STATUS_0_MEAN          0
STATUS_1_MEAN          0
STATUS_2_MEAN          0
STATUS_3_MEAN          0
STATUS_4_MEAN          0
STATUS_5_MEAN          0
STATUS_C_MEAN          0
STATUS_X_MEAN          0
STATUS_nan_MEAN        0
NEW_FLAG_0_MEAN        0
NEW_FLAG_1_MEAN        0
NEW_FLAG_nan_MEAN      0
dtype: int64

In [60]:
bb_agg.head()

,SK_ID_BUREAU_,MONTHS_BALANCE_MIN,MONTHS_BALANCE_MAX,MONTHS_BALANCE_SIZE,STATUS_0_MEAN,STATUS_1_MEAN,STATUS_2_MEAN,STATUS_3_MEAN,STATUS_4_MEAN,STATUS_5_MEAN,STATUS_C_MEAN,STATUS_X_MEAN,STATUS_nan_MEAN,NEW_FLAG_0_MEAN,NEW_FLAG_1_MEAN,NEW_FLAG_nan_MEAN
0,5001709,-96,0,97,0.000000,0.0,0.0,0.0,0.0,0.0,0.886598,0.113402,0.0,0.886598,0.0,0.0
1,5001710,-82,0,83,0.060241,0.0,0.0,0.0,0.0,0.0,0.578313,0.361446,0.0,0.638554,0.0,0.0
2,5001711,-3,0,4,0.750000,0.0,0.0,0.0,0.0,0.0,0.000000,0.250000,0.0,0.750000,0.0,0.0
3,5001712,-18,0,19,0.526316,0.0,0.0,0.0,0.0,0.0,0.473684,0.000000,0.0,1.000000,0.0,0.0
4,5001713,-21,0,22,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.0,0.000000,0.0,0.0


In [61]:
bb_agg.to_csv("/content/drive/MyDrive/p7/data/bb_agg_final.csv", index=False)

## **bureau**



In [61]:
bu= pd.read_csv('/content/drive/MyDrive/p7/data/bureau.csv', sep=',', encoding='utf-8', low_memory=False)

In [63]:
bu.shape

(1716428, 17)

In [62]:
#from the EDA we saw some erroneous values in DAYS Fields, we will remove those
#there are some loans which ended about very long ago, around 100 years ago.
#Thus we will only keep those loans which have ended in past 50 years.
bu=bu.loc[bu['DAYS_CREDIT_ENDDATE'] > -50 *365] 

#there is also a feature which tells about the number of days ago the Credit Report Came
bu= bu.loc[bu['DAYS_CREDIT_UPDATE'] > -50 *365] 

In [65]:
bu.shape

(1610646, 17)

**Feature engineering**

In [63]:
# Number of past loans per customer.*
grp = bu[['SK_ID_CURR', 'DAYS_CREDIT']].groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT'].count().reset_index().rename(index=str, columns={'DAYS_CREDIT': 'NEW_BUREAU_LOAN_COUNT'})
bu = bu.merge(grp, on = ['SK_ID_CURR'], how = 'left')

# Number of types of past loans per customer.*
grp = bu[['SK_ID_CURR', 'CREDIT_TYPE']].groupby(by = ['SK_ID_CURR'])['CREDIT_TYPE'].nunique().reset_index().rename(index=str, columns={'CREDIT_TYPE': 'NEW_BUREAU_LOAN_TYPES'})
bu = bu.merge(grp, on = ['SK_ID_CURR'], how = 'left')

In [67]:
bu.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,NEW_BUREAU_LOAN_COUNT,NEW_BUREAU_LOAN_TYPES
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.00,NaN,0.0,Consumer credit,-131,NaN,10,3
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.00,NaN,0.0,Credit card,-20,NaN,10,3
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,10,3
3,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,10,3
4,215354,5714467,Active,currency 1,-273,0,27460.0,NaN,0.0,0,180000.0,71017.38,108982.62,0.0,Credit card,-31,NaN,10,3


In [64]:
 # Create a new dummy column for whether CREDIT is ACTIVE OR CLOED *
bu['CREDIT_ACTIVE_BINARY'] = bu['CREDIT_ACTIVE']

def f(x):
    if x == 'Closed':
        y = 0
    else:
        y = 1    
    return y

bu['CREDIT_ACTIVE_BINARY'] = bu.apply(lambda x: f(x.CREDIT_ACTIVE), axis = 1)

In [65]:
# Calculate mean number of loans that are ACTIVE per CUSTOMER *
grp = bu.groupby(by = ['SK_ID_CURR'])['CREDIT_ACTIVE_BINARY'].mean().reset_index().rename(index=str, columns={'CREDIT_ACTIVE_BINARY': 'NEW_ACTIVE_LOANS_PERCENTAGE'})
bu = bu.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del bu['CREDIT_ACTIVE_BINARY']

In [70]:
bu.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,NEW_BUREAU_LOAN_COUNT,NEW_BUREAU_LOAN_TYPES,NEW_ACTIVE_LOANS_PERCENTAGE
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.00,NaN,0.0,Consumer credit,-131,NaN,10,3,0.5
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.00,NaN,0.0,Credit card,-20,NaN,10,3,0.5
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,10,3,0.5
3,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,10,3,0.5
4,215354,5714467,Active,currency 1,-273,0,27460.0,NaN,0.0,0,180000.0,71017.38,108982.62,0.0,Credit card,-31,NaN,10,3,0.5


In [78]:
bu.shape

(1610646, 20)

In [66]:
# Calculating Loan Activated and Closed Total Numbers and Rates
temp_bu = bu[['SK_ID_CURR', 'CREDIT_ACTIVE']]
temp_bu = pd.get_dummies(temp_bu)
temp_bu = temp_bu.groupby('SK_ID_CURR').agg({'CREDIT_ACTIVE_Active': 'sum', 'CREDIT_ACTIVE_Closed': 'sum'})

temp_bu.columns = ['CREDIT_ACTIVE_Active_Count', 'CREDIT_ACTIVE_Closed_Count']

temp_bu['CREDIT_ACTIVE_Active_ratio'] = temp_bu['CREDIT_ACTIVE_Active_Count'] / (
            temp_bu['CREDIT_ACTIVE_Active_Count'] + temp_bu['CREDIT_ACTIVE_Closed_Count'])
temp_bu['CREDIT_ACTIVE_Closed_ratio'] = temp_bu['CREDIT_ACTIVE_Closed_Count'] / (
            temp_bu['CREDIT_ACTIVE_Active_Count'] + temp_bu['CREDIT_ACTIVE_Closed_Count'])
bu = bu.merge(temp_bu, on=['SK_ID_CURR'], how='left')

In [67]:
# Is the loan closed early for Active and Closed Loans?*
bu.loc[(bu['CREDIT_ACTIVE'] == 'Active') & (bu['DAYS_CREDIT_ENDDATE'] < 0), 'NEW_EARLY_ACTİVE'] = 1
bu.loc[(bu['CREDIT_ACTIVE'] == 'Closed') & (
            abs(bu['DAYS_CREDIT_ENDDATE']) < abs(bu['DAYS_ENDDATE_FACT'])), 'NEW_EARLY_CLOSED'] = 1

In [68]:
# How many different types of loans has the person received?
temp_bu = bu[['SK_ID_CURR', 'CREDIT_TYPE']].groupby(by=['SK_ID_CURR'])[
                                                   'CREDIT_TYPE'].nunique().reset_index().rename(index=str, columns={'CREDIT_TYPE': 'BUREAU_LOAN_TYPES'})
bu = bu.merge(temp_bu, on=['SK_ID_CURR'], how='left')

# loan ratio
bu['NEW_DEPT_RATİO'] = bu['AMT_CREDIT_SUM_DEBT'] / (bu['AMT_CREDIT_SUM'] + 1)

In [69]:
bu.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,NEW_BUREAU_LOAN_COUNT,NEW_BUREAU_LOAN_TYPES,NEW_ACTIVE_LOANS_PERCENTAGE,CREDIT_ACTIVE_Active_Count,CREDIT_ACTIVE_Closed_Count,CREDIT_ACTIVE_Active_ratio,CREDIT_ACTIVE_Closed_ratio,NEW_EARLY_ACTİVE,NEW_EARLY_CLOSED,BUREAU_LOAN_TYPES,NEW_DEPT_RATİO
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.00,NaN,0.0,Consumer credit,-131,NaN,10,3,0.5,5,5,0.5,0.5,NaN,NaN,3,0.000000
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.00,NaN,0.0,Credit card,-20,NaN,10,3,0.5,5,5,0.5,0.5,NaN,NaN,3,0.761517
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,10,3,0.5,5,5,0.5,0.5,NaN,NaN,3,NaN
3,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,10,3,0.5,5,5,0.5,0.5,NaN,NaN,3,NaN
4,215354,5714467,Active,currency 1,-273,0,27460.0,NaN,0.0,0,180000.0,71017.38,108982.62,0.0,Credit card,-31,NaN,10,3,0.5,5,5,0.5,0.5,NaN,NaN,3,0.394539


In [70]:
bu.shape

(1610646, 28)

In [85]:
bu['CREDIT_TYPE'].unique()

array(['Consumer credit', 'Credit card', 'Mortgage', 'Car loan',
       'Microloan', 'Loan for working capital replenishment',
       'Loan for business development', 'Real estate loan',
       'Unknown type of loan', 'Another type of loan',
       'Cash loan (non-earmarked)', 'Loan for the purchase of equipment',
       'Mobile operator loan', 'Interbank credit',
       'Loan for purchase of shares (margin lending)'], dtype=object)

In [71]:
# Replacing Credit Types with 'others'
credit_type = ['Loan for working capital replenishment',
                'Loan for business development', 'Real estate loan',
                'Unknown type of loan', 'Another type of loan',
                'Cash loan (non-earmarked)', 'Loan for the purchase of equipment',
                'Mobile operator loan', 'Interbank credit',
                'Loan for purchase of shares (margin lending)']
bu['CREDIT_TYPE'].replace(credit_type, 'others', inplace=True)

In [72]:
# Monthly Payout Rate
bu['NEW_AMT_ANNUITY_RATİO'] = bu['AMT_ANNUITY'] / bu['AMT_CREDIT_SUM']

# Is the credit update new?
bu['NEWS_DAYS_CREDIT_UPDATE'] = bu['DAYS_CREDIT_UPDATE'].apply(lambda x: 'old' if x < -90 else 'new')


In [73]:
# we  have dropped this variable
bu.drop('CREDIT_CURRENCY', inplace=True, axis=1)

In [74]:
del temp_bu
gc.collect()

219

In [75]:
# Categorical features with One-Hot encode

bu, bu_cat = one_hot_encoder(bu, nan_as_category=True)

In [76]:
bu_merge = bu.join(bb_agg, how='left', on='SK_ID_BUREAU')
bu_merge.drop(['SK_ID_BUREAU'], axis=1, inplace=True)

In [77]:
bu_merge.head()

,SK_ID_CURR,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,NEW_BUREAU_LOAN_COUNT,NEW_BUREAU_LOAN_TYPES,NEW_ACTIVE_LOANS_PERCENTAGE,CREDIT_ACTIVE_Active_Count,CREDIT_ACTIVE_Closed_Count,CREDIT_ACTIVE_Active_ratio,CREDIT_ACTIVE_Closed_ratio,NEW_EARLY_ACTİVE,NEW_EARLY_CLOSED,BUREAU_LOAN_TYPES,NEW_DEPT_RATİO,NEW_AMT_ANNUITY_RATİO,CREDIT_ACTIVE_Active,CREDIT_ACTIVE_Bad debt,CREDIT_ACTIVE_Closed,CREDIT_ACTIVE_Sold,CREDIT_ACTIVE_nan,CREDIT_TYPE_Car loan,CREDIT_TYPE_Consumer credit,CREDIT_TYPE_Credit card,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_others,CREDIT_TYPE_nan,NEWS_DAYS_CREDIT_UPDATE_new,NEWS_DAYS_CREDIT_UPDATE_old,NEWS_DAYS_CREDIT_UPDATE_nan,SK_ID_BUREAU_,MONTHS_BALANCE_MIN,MONTHS_BALANCE_MAX,MONTHS_BALANCE_SIZE,STATUS_0_MEAN,STATUS_1_MEAN,STATUS_2_MEAN,STATUS_3_MEAN,STATUS_4_MEAN,STATUS_5_MEAN,STATUS_C_MEAN,STATUS_X_MEAN,STATUS_nan_MEAN,NEW_FLAG_0_MEAN,NEW_FLAG_1_MEAN,NEW_FLAG_nan_MEAN
0,215354,-497,0,-153.0,-153.0,NaN,0,91323.0,0.00,NaN,0.0,-131,NaN,10,3,0.5,5,5,0.5,0.5,NaN,NaN,3,0.000000,NaN,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,215354,-208,0,1075.0,NaN,NaN,0,225000.0,171342.00,NaN,0.0,-20,NaN,10,3,0.5,5,5,0.5,0.5,NaN,NaN,3,0.761517,NaN,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,215354,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,-16,NaN,10,3,0.5,5,5,0.5,0.5,NaN,NaN,3,NaN,NaN,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,215354,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,-21,NaN,10,3,0.5,5,5,0.5,0.5,NaN,NaN,3,NaN,NaN,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,215354,-273,0,27460.0,NaN,0.0,0,180000.0,71017.38,108982.62,0.0,-31,NaN,10,3,0.5,5,5,0.5,0.5,NaN,NaN,3,0.394539,NaN,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
bu_cat

['CREDIT_ACTIVE_Active',
 'CREDIT_ACTIVE_Bad debt',
 'CREDIT_ACTIVE_Closed',
 'CREDIT_ACTIVE_Sold',
 'CREDIT_ACTIVE_nan',
 'CREDIT_TYPE_Car loan',
 'CREDIT_TYPE_Consumer credit',
 'CREDIT_TYPE_Credit card',
 'CREDIT_TYPE_Microloan',
 'CREDIT_TYPE_Mortgage',
 'CREDIT_TYPE_others',
 'CREDIT_TYPE_nan',
 'NEWS_DAYS_CREDIT_UPDATE_new',
 'NEWS_DAYS_CREDIT_UPDATE_old',
 'NEWS_DAYS_CREDIT_UPDATE_nan']

In [78]:
# Bureau and bureau_balance numeric features
num_aggregations = {
    'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
    'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
    'DAYS_CREDIT_UPDATE': ['mean'],
    'CREDIT_DAY_OVERDUE': ['max', 'mean'],
    'AMT_CREDIT_MAX_OVERDUE': ['mean'],
    'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
    'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
    'AMT_CREDIT_SUM_OVERDUE': ['mean'],
    'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
    'AMT_ANNUITY': ['max', 'mean'],
    'CNT_CREDIT_PROLONG': ['sum'],
    'MONTHS_BALANCE_MIN': ['min'],
    'MONTHS_BALANCE_MAX': ['max'],
    'MONTHS_BALANCE_SIZE': ['mean', 'sum'],
    "CREDIT_ACTIVE_Active_Count": ["mean"],
    "CREDIT_ACTIVE_Closed_Count": ["mean"],
    "CREDIT_ACTIVE_Active_ratio": ["mean"],
    "NEW_EARLY_ACTİVE": ['mean'],
    "NEW_EARLY_CLOSED": ['mean'],
    "NEW_BUREAU_LOAN_TYPES": ['mean'],
    "NEW_DEPT_RATİO": ['max', 'mean'],
    "NEW_AMT_ANNUITY_RATİO": ['max', 'mean']
}

for col in bb_cat:
    num_aggregations[col + "_MEAN"] = ['mean']

# Bureau and bureau_balance categorical features
cat_aggregations = {}
for cat in bu_cat: cat_aggregations[cat] = ['mean']
for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']

bu_agg = bu_merge.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
bu_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bu_agg.columns.tolist()])

In [80]:
bu_agg.head()

,BURO_DAYS_CREDIT_MIN,BURO_DAYS_CREDIT_MAX,BURO_DAYS_CREDIT_MEAN,BURO_DAYS_CREDIT_VAR,BURO_DAYS_CREDIT_ENDDATE_MIN,BURO_DAYS_CREDIT_ENDDATE_MAX,BURO_DAYS_CREDIT_ENDDATE_MEAN,BURO_DAYS_CREDIT_UPDATE_MEAN,BURO_CREDIT_DAY_OVERDUE_MAX,BURO_CREDIT_DAY_OVERDUE_MEAN,BURO_AMT_CREDIT_MAX_OVERDUE_MEAN,BURO_AMT_CREDIT_SUM_MAX,BURO_AMT_CREDIT_SUM_MEAN,BURO_AMT_CREDIT_SUM_SUM,BURO_AMT_CREDIT_SUM_DEBT_MAX,BURO_AMT_CREDIT_SUM_DEBT_MEAN,BURO_AMT_CREDIT_SUM_DEBT_SUM,BURO_AMT_CREDIT_SUM_OVERDUE_MEAN,BURO_AMT_CREDIT_SUM_LIMIT_MEAN,BURO_AMT_CREDIT_SUM_LIMIT_SUM,BURO_AMT_ANNUITY_MAX,BURO_AMT_ANNUITY_MEAN,BURO_CNT_CREDIT_PROLONG_SUM,BURO_MONTHS_BALANCE_MIN_MIN,BURO_MONTHS_BALANCE_MAX_MAX,BURO_MONTHS_BALANCE_SIZE_MEAN,BURO_MONTHS_BALANCE_SIZE_SUM,BURO_CREDIT_ACTIVE_Active_Count_MEAN,BURO_CREDIT_ACTIVE_Closed_Count_MEAN,BURO_CREDIT_ACTIVE_Active_ratio_MEAN,BURO_NEW_EARLY_ACTİVE_MEAN,BURO_NEW_EARLY_CLOSED_MEAN,BURO_NEW_BUREAU_LOAN_TYPES_MEAN,BURO_NEW_DEPT_RATİO_MAX,BURO_NEW_DEPT_RATİO_MEAN,BURO_NEW_AMT_ANNUITY_RATİO_MAX,BURO_NEW_AMT_ANNUITY_RATİO_MEAN,BURO_STATUS_0_MEAN_MEAN,BURO_STATUS_1_MEAN_MEAN,BURO_STATUS_2_MEAN_MEAN,BURO_STATUS_3_MEAN_MEAN,BURO_STATUS_4_MEAN_MEAN,BURO_STATUS_5_MEAN_MEAN,BURO_STATUS_C_MEAN_MEAN,BURO_STATUS_X_MEAN_MEAN,BURO_STATUS_nan_MEAN_MEAN,BURO_NEW_FLAG_0_MEAN_MEAN,BURO_NEW_FLAG_1_MEAN_MEAN,BURO_NEW_FLAG_nan_MEAN_MEAN,BURO_CREDIT_ACTIVE_Active_MEAN,BURO_CREDIT_ACTIVE_Bad debt_MEAN,BURO_CREDIT_ACTIVE_Closed_MEAN,BURO_CREDIT_ACTIVE_Sold_MEAN,BURO_CREDIT_ACTIVE_nan_MEAN,BURO_CREDIT_TYPE_Car loan_MEAN,BURO_CREDIT_TYPE_Consumer credit_MEAN,BURO_CREDIT_TYPE_Credit card_MEAN,BURO_CREDIT_TYPE_Microloan_MEAN,BURO_CREDIT_TYPE_Mortgage_MEAN,BURO_CREDIT_TYPE_others_MEAN,BURO_CREDIT_TYPE_nan_MEAN,BURO_NEWS_DAYS_CREDIT_UPDATE_new_MEAN,BURO_NEWS_DAYS_CREDIT_UPDATE_old_MEAN,BURO_NEWS_DAYS_CREDIT_UPDATE_nan_MEAN
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,-1572,-49,-735.000000,240043.666667,-1329.0,1778.0,82.428571,-93.142857,0,0.0,NaN,378000.0,207623.571429,1453365.0,373239.0,85240.928571,596686.5,0.0,0.0,0.0,10822.5,3545.357143,0,NaN,NaN,NaN,0.0,3.0,4.0,0.428571,NaN,1.0,1.0,0.987402,0.282517,0.055627,0.013996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.428571,0.0,0.571429,0.0,0.0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.428571,0.571429,0.0
100002,-1437,-645,-1068.833333,64488.966667,-1072.0,780.0,-349.000000,-654.666667,0,0.0,2091.16125,450000.0,138844.500000,833067.0,245781.0,81927.000000,245781.0,0.0,0.0,0.0,0.0,0.000000,0,NaN,NaN,NaN,0.0,1.0,5.0,0.166667,NaN,1.0,2.0,0.546179,0.182060,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.166667,0.0,0.833333,0.0,0.0,0.0,0.666667,0.333333,0.0,0.0,0.0,0.0,0.333333,0.666667,0.0
100003,-2586,-606,-1400.750000,827783.583333,-2434.0,1216.0,-544.500000,-816.000000,0,0.0,0.00000,810000.0,254350.125000,1017400.5,0.0,0.000000,0.0,0.0,202500.0,810000.0,NaN,NaN,0,NaN,NaN,NaN,0.0,1.0,3.0,0.250000,NaN,1.0,2.0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.250000,0.0,0.750000,0.0,0.0,0.0,0.500000,0.500000,0.0,0.0,0.0,0.0,0.250000,0.750000,0.0
100004,-1326,-408,-867.000000,421362.000000,-595.0,-382.0,-488.500000,-532.000000,0,0.0,0.00000,94537.8,94518.900000,189037.8,0.0,0.000000,0.0,0.0,0.0,0.0,NaN,NaN,0,NaN,NaN,NaN,0.0,0.0,2.0,0.000000,NaN,1.0,1.0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.0,1.000000,0.0,0.0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,1.000000,0.0
100005,-373,-62,-190.666667,26340.333333,-128.0,1324.0,439.333333,-54.333333,0,0.0,0.00000,568800.0,219042.000000,657126.0,543087.0,189469.500000,568408.5,0.0,0.0,0.0,4261.5,1420.500000,0,NaN,NaN,NaN,0.0,2.0,1.0,0.666667,NaN,NaN,2.0,0.954793,0.601246,0.142879,0.047626,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.666667,0.0,0.333333,0.0,0.0,0.0,0.666667,0.333333,0.0,0.0,0.0,0.0,0.666667,0.333333,0.0


In [87]:
# Bureau: Active credits - using only numerical aggregations
active = bu_merge[bu_merge['CREDIT_ACTIVE_Active'] == 1]
active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations).reset_index()
active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
bu_agg_final = bu_agg.join(active_agg, how='left', on='SK_ID_CURR')

In [82]:
del active, active_agg
gc.collect()

489

In [88]:
 # Bureau: Closed credits - using only numerical aggregations
closed = bu_merge[bu_merge['CREDIT_ACTIVE_Closed'] == 1]
closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
bu_agg_final = bu_agg_final.join(closed_agg, how='left', on='SK_ID_CURR')

In [84]:
del closed, closed_agg, bureau, bb_agg
gc.collect()

NameError: ignored

In [110]:
bu_agg_final.to_csv("/content/drive/MyDrive/p7/data/bu_agg_final.csv", index=False)

### **previous_application.csv**

* application of client's previous loan in home credit
* Info about the previous loan parameters and client info at time of previous app
* one row per perivoius app 

In [119]:
df_prev= pd.read_csv('/content/drive/MyDrive/p7/data/previous_application.csv', encoding='utf-8',low_memory=False)

In [120]:
df_prev.shape

(1670214, 37)

In [121]:
cat_cols = [col for col in df_prev.columns if df_prev[col].dtypes == 'O']
num_cols = [col for col in df_prev.columns if df_prev[col].dtypes != 'O']

In [122]:
 # days 365243 values to nan
df_prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace=True)
df_prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace=True)
df_prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace=True)
df_prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace=True)
df_prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace=True)

# XNA, XAP to nan for cat_cols.
na = ['XNA', 'XAP']
for col in cat_cols:
    for n in na:
        df_prev.loc[df_prev[col] == n, col] = np.nan

# delete columns columns that do not contain information or missing values over 80 percent of the entire data
del_cols = ['RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_FIRST_DRAWING',
            'NAME_CASH_LOAN_PURPOSE', 'CODE_REJECT_REASON', 'FLAG_LAST_APPL_PER_CONTRACT',
            'NFLAG_LAST_APPL_IN_DAY', 'SELLERPLACE_AREA']
df_prev.drop(del_cols, axis=1, inplace=True)


In [123]:
# Feature Engineering
# X-sell approved & Walk-in Approved
df_prev['NEW_X_SELL_APPROVED'] = 0
df_prev.loc[(df_prev['NAME_PRODUCT_TYPE'] == 'x-sell') &
            (df_prev['NAME_CONTRACT_STATUS'] == 'Approved'), 'NEW_X_SELL_APPROVED'] = 1
df_prev['NEW_WALK_IN_APPROVED'] = 0
df_prev.loc[(df_prev['NAME_PRODUCT_TYPE'] == 'walk-in') &
            (df_prev['NAME_CONTRACT_STATUS'] == 'Approved'), 'NEW_WALK_IN_APPROVED'] = 1

# Customer status approved
df_prev['NEW_REPEATER_APPROVED'] = 0
df_prev.loc[(df_prev['NAME_CLIENT_TYPE'] == 'Repeater') &
            (df_prev['NAME_CONTRACT_STATUS'] == 'Approved'), 'NEW_REPEATER_APPROVED'] = 1
df_prev['NEW_NEWCUST_APPROVED'] = 0
df_prev.loc[(df_prev['NAME_CLIENT_TYPE'] == 'New') &
            (df_prev['NAME_CONTRACT_STATUS'] == 'Approved'), 'NEW_NEWCUST_APPROVED'] = 1
df_prev['NEW_REFRESHED_APPROVED'] = 0
df_prev.loc[(df_prev['NAME_CLIENT_TYPE'] == 'Refreshed') &
            (df_prev['NAME_CONTRACT_STATUS'] == 'Approved'), 'NEW_REFRESHED_APPROVED'] = 1

In [125]:
# Purpose of application approved
df_prev['NEW_CARDS_APPROVED'] = 0
df_prev.loc[(df_prev['NAME_PORTFOLIO'] == 'Cards') &
            (df_prev['NAME_CONTRACT_STATUS'] == 'Approved'), 'NEW_CARDS_APPROVED'] = 1
df_prev['NEW_CASH_APPROVED'] = 0
df_prev.loc[(df_prev['NAME_PORTFOLIO'] == 'Cash') &
            (df_prev['NAME_CONTRACT_STATUS'] == 'Approved'), 'NEW_CASH_APPROVED'] = 1
df_prev['NEW_POS_APPROVED'] = 0
df_prev.loc[(df_prev['NAME_PORTFOLIO'] == 'POS') &
            (df_prev['NAME_CONTRACT_STATUS'] == 'Approved'), 'NEW_POS_APPROVED'] = 1

  # Interest approved
df_prev['NEW_HIGH_APPROVED'] = 0
df_prev.loc[(df_prev['NAME_YIELD_GROUP'] == 'high') &
            (df_prev['NAME_CONTRACT_STATUS'] == 'Approved'), 'NEW_HIGH_APPROVED'] = 1
df_prev['NEW_MIDDLE_APPROVED'] = 0
df_prev.loc[(df_prev['NAME_YIELD_GROUP'] == 'middle') &
            (df_prev['NAME_CONTRACT_STATUS'] == 'Approved'), 'NEW_MIDDLE_APPROVED'] = 1
df_prev['NEW_LOWACTION_APPROVED'] = 0
df_prev.loc[(df_prev['NAME_YIELD_GROUP'] == 'low_action') &
            (df_prev['NAME_CONTRACT_STATUS'] == 'Approved'), 'NEW_LOWACTION_APPROVED'] = 1
df_prev['NEW_LOWNORMAL_APPROVED'] = 0
df_prev.loc[(df_prev['NAME_YIELD_GROUP'] == 'low_normal') &
            (df_prev['NAME_CONTRACT_STATUS'] == 'Approved'), 'NEW_LOWNORMAL_APPROVED'] = 1

# Application hour convert to categorical
df_prev.loc[(df_prev['HOUR_APPR_PROCESS_START'] >= 0) &
            (df_prev['HOUR_APPR_PROCESS_START'] <= 6), 'NEW_APP_DAY_TIME'] = 'night'
df_prev.loc[(df_prev['HOUR_APPR_PROCESS_START'] > 6) &
            (df_prev['HOUR_APPR_PROCESS_START'] <= 12), 'NEW_APP_DAY_TIME'] = 'morning'
df_prev.loc[(df_prev['HOUR_APPR_PROCESS_START'] > 12) &
            (df_prev['HOUR_APPR_PROCESS_START'] <= 18), 'NEW_APP_DAY_TIME'] = 'afternoon'
df_prev.loc[(df_prev['HOUR_APPR_PROCESS_START'] > 18) &
            (df_prev['HOUR_APPR_PROCESS_START'] < 24), 'NEW_APP_DAY_TIME'] = 'evening'
df_prev.drop('HOUR_APPR_PROCESS_START', axis=1, inplace=True)

# Client apply with someone
df_prev.loc[df_prev['NAME_TYPE_SUITE'] == 'Unaccompanied', 'NEW_ACCOMPANIED'] = 0
df_prev.loc[df_prev['NAME_TYPE_SUITE'] != 'Unaccompanied', 'NEW_ACCOMPANIED'] = 1
df_prev.loc[df_prev['NAME_TYPE_SUITE'].isnull(), 'NEW_ACCOMPANIED'] = np.nan
df_prev.drop('NAME_TYPE_SUITE', axis=1, inplace=True)

    # credit requested / credit given ratio
df_prev['NEW_APP_CREDIT_RATIO'] = df_prev['AMT_APPLICATION'].div(df_prev['AMT_CREDIT']).replace(np.inf, 0)

# loan installment / credit amount ratio
df_prev['NEW_ANNUITY_CREDIT_RATIO'] = df_prev['AMT_ANNUITY'] / df_prev['AMT_CREDIT']

# credit amount / goods price ratio
df_prev['NEW_CREDIT_GOODS_RATIO'] = df_prev['AMT_CREDIT'].div(df_prev['AMT_GOODS_PRICE']).replace(np.inf, 0)

# interest amount
df_prev['NEW_AMT_INTEREST'] = df_prev['CNT_PAYMENT'] * df_prev['AMT_ANNUITY'] - df_prev['AMT_CREDIT']
# interest ratio
df_prev['NEW_INTEREST_RATIO'] = df_prev['NEW_AMT_INTEREST'] / df_prev['AMT_CREDIT']
# needed amount / credit amount (belki silinir)
df_prev['NEW_AMT_NEEDED_CREDIT_RATIO'] = (df_prev['AMT_GOODS_PRICE'] - df_prev['AMT_DOWN_PAYMENT']) / \
                                          df_prev['AMT_CREDIT']

# risk assessment via NEW_CREDIT_GOODS_RATIO
df_prev.loc[df_prev['NEW_CREDIT_GOODS_RATIO'] >= 0.80, 'NEW_CREDIT_GOODS_RISK'] = 1
df_prev.loc[df_prev['NEW_CREDIT_GOODS_RATIO'] < 0.80, 'NEW_CREDIT_GOODS_RISK'] = 0

# risk to approved
df_prev['NEW_RISK_APPROVED'] = 0
df_prev.loc[(df_prev['NEW_CREDIT_GOODS_RISK'] == 1) &
            (df_prev['NAME_CONTRACT_STATUS'] == 'Approved'), 'NEW_RISK_APPROVED'] = 1

  # non risk to approved
df_prev['NEW_NONRISK_APPROVED'] = 0
df_prev.loc[(df_prev['NEW_CREDIT_GOODS_RISK'] == 0) &
            (df_prev['NAME_CONTRACT_STATUS'] == 'Approved'), 'NEW_NONRISK_APPROVED'] = 1

# Application weekdays cycle encoding
df_prev['WEEKDAY_APPR_PROCESS_START'] = df_prev['WEEKDAY_APPR_PROCESS_START'].map({
    'MONDAY': 1, 'TUESDAY': 2, 'WEDNESDAY': 3, 'THURSDAY': 4, 'FRIDAY': 5, 'SATURDAY': 6, 'SUNDAY': 7})
df_prev['NEW_WEEKDAY_SIN'] = np.sin(2 * np.pi * df_prev['WEEKDAY_APPR_PROCESS_START'] / 7)
df_prev['NEW_WEEKDAY_COS'] = np.cos(2 * np.pi * df_prev['WEEKDAY_APPR_PROCESS_START'] / 7)
df_prev.drop('WEEKDAY_APPR_PROCESS_START', axis=1, inplace=True)
        

In [127]:
# Rare encoding
a = ['Auto Accessories', 'Jewelry', 'Homewares', 'Medical Supplies', 'Vehicles', 'Sport and Leisure',
      'Gardening', 'Other', 'Office Appliances', 'Tourism', 'Medicine', 'Direct Sales', 'Fitness',
      'Additional Service', 'Education', 'Weapon', 'Insurance', 'House Construction', 'Animals']
df_prev["NAME_GOODS_CATEGORY"] = df_prev["NAME_GOODS_CATEGORY"].replace(a, 'others')

b = ['Channel of corporate sales', 'Car dealer']
df_prev["CHANNEL_TYPE"] = df_prev["CHANNEL_TYPE"].replace(b, 'Other_Channel')

c = ['Auto technology', 'Jewelry', 'MLM partners', 'Tourism']
df_prev["NAME_SELLER_INDUSTRY"] = df_prev["NAME_SELLER_INDUSTRY"].replace(c, 'Others')

d = ['Non-cash from your account', 'Cashless from the account of the employer']
df_prev["NAME_PAYMENT_TYPE"] = df_prev["NAME_SELLER_INDUSTRY"].replace(d, 'Others')

In [129]:
# One hot encoder
new_df_prev, new_cat_cols = one_hot_encoder(df_prev, nan_as_category= True)

In [130]:
# Getting to all the cat cols
origin_bin_cols = [col for col in df_prev.columns if (df_prev[col].dtypes != 'O') & (df_prev[col].nunique() == 2)]
all_cat_cols = new_cat_cols + origin_bin_cols

In [131]:
# Getting to the num cols
# x_cols = ['SK_ID_PREV','SK_ID_CURR', 'DAYS_FIRST_DUE','DAYS_LAST_DUE_1ST_VERSION','DAYS_LAST_DUE','DAYS_TERMINATION']
# new_num_cols = [col for col in new_df.columns if (col not in all_binary_cols) and (col not in x_cols)]
# num_aggregations = {}
# for num in new_num_cols:
# num_aggregations[num] = ['min', 'max', 'mean', 'median']

# Previous app num features
num_aggregations = {
    'AMT_ANNUITY': ['min', 'max', 'mean', 'median'],
    'AMT_APPLICATION': ['min', 'max', 'mean', 'median'],
    'AMT_CREDIT': ['min', 'max', 'mean', 'median'],
    'AMT_DOWN_PAYMENT': ['min', 'max', 'mean', 'median'],
    'AMT_GOODS_PRICE': ['min', 'max', 'mean', 'median'],
    'RATE_DOWN_PAYMENT': ['min', 'max', 'mean', 'median'],
    'DAYS_DECISION': ['min', 'max', 'mean', 'median'],
    'CNT_PAYMENT': ['min', 'max', 'mean', 'median'],
    'NEW_APP_CREDIT_RATIO': ['min', 'max', 'mean', 'median'],
    'NEW_ANNUITY_CREDIT_RATIO': ['min', 'max', 'mean', 'median'],
    'NEW_CREDIT_GOODS_RATIO': ['min', 'max', 'mean', 'median'],
    'NEW_AMT_INTEREST': ['min', 'max', 'mean', 'median'],
            'NEW_INTEREST_RATIO': ['min', 'max', 'mean', 'median'],
    'NEW_AMT_NEEDED_CREDIT_RATIO': ['min', 'max', 'mean', 'median'],
    'NEW_WEEKDAY_SIN': ['min', 'max', 'mean', 'median'],
    'NEW_WEEKDAY_COS': ['min', 'max', 'mean', 'median']}

# Previous app cat features
cat_aggregations = {}
for cat in all_cat_cols:
    cat_aggregations[cat] = ['mean']

final_prev_df = new_df_prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
final_prev_df.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in final_prev_df.columns.tolist()])


In [132]:
# Approved App - only num features
approved = new_df_prev[new_df_prev['NAME_CONTRACT_STATUS_Approved'] == 1]
approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
approved_agg.columns = pd.Index(
    ['PREV_APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
final_prev_df = final_prev_df.join(approved_agg, how='left', on='SK_ID_CURR')

In [133]:
# refused App - only numerical features
refused = new_df_prev[new_df_prev['NAME_CONTRACT_STATUS_Refused'] == 1]
refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
refused_agg.columns = pd.Index(['PREV_REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
final_prev_df = final_prev_df.join(refused_agg, how='left', on='SK_ID_CURR')

In [134]:
del refused, refused_agg, approved, approved_agg, new_df_prev
gc.collect()

1792

In [136]:
final_prev_df.head()

,PREV_AMT_ANNUITY_MIN,PREV_AMT_ANNUITY_MAX,PREV_AMT_ANNUITY_MEAN,PREV_AMT_ANNUITY_MEDIAN,PREV_AMT_APPLICATION_MIN,PREV_AMT_APPLICATION_MAX,PREV_AMT_APPLICATION_MEAN,PREV_AMT_APPLICATION_MEDIAN,PREV_AMT_CREDIT_MIN,PREV_AMT_CREDIT_MAX,PREV_AMT_CREDIT_MEAN,PREV_AMT_CREDIT_MEDIAN,PREV_AMT_DOWN_PAYMENT_MIN,PREV_AMT_DOWN_PAYMENT_MAX,PREV_AMT_DOWN_PAYMENT_MEAN,PREV_AMT_DOWN_PAYMENT_MEDIAN,PREV_AMT_GOODS_PRICE_MIN,PREV_AMT_GOODS_PRICE_MAX,PREV_AMT_GOODS_PRICE_MEAN,PREV_AMT_GOODS_PRICE_MEDIAN,PREV_RATE_DOWN_PAYMENT_MIN,PREV_RATE_DOWN_PAYMENT_MAX,PREV_RATE_DOWN_PAYMENT_MEAN,PREV_RATE_DOWN_PAYMENT_MEDIAN,PREV_DAYS_DECISION_MIN,PREV_DAYS_DECISION_MAX,PREV_DAYS_DECISION_MEAN,PREV_DAYS_DECISION_MEDIAN,PREV_CNT_PAYMENT_MIN,PREV_CNT_PAYMENT_MAX,PREV_CNT_PAYMENT_MEAN,PREV_CNT_PAYMENT_MEDIAN,PREV_NEW_APP_CREDIT_RATIO_MIN,PREV_NEW_APP_CREDIT_RATIO_MAX,PREV_NEW_APP_CREDIT_RATIO_MEAN,PREV_NEW_APP_CREDIT_RATIO_MEDIAN,PREV_NEW_ANNUITY_CREDIT_RATIO_MIN,PREV_NEW_ANNUITY_CREDIT_RATIO_MAX,PREV_NEW_ANNUITY_CREDIT_RATIO_MEAN,PREV_NEW_ANNUITY_CREDIT_RATIO_MEDIAN,...,PREV_REFUSED_DAYS_DECISION_MIN,PREV_REFUSED_DAYS_DECISION_MAX,PREV_REFUSED_DAYS_DECISION_MEAN,PREV_REFUSED_DAYS_DECISION_MEDIAN,PREV_REFUSED_CNT_PAYMENT_MIN,PREV_REFUSED_CNT_PAYMENT_MAX,PREV_REFUSED_CNT_PAYMENT_MEAN,PREV_REFUSED_CNT_PAYMENT_MEDIAN,PREV_REFUSED_NEW_APP_CREDIT_RATIO_MIN,PREV_REFUSED_NEW_APP_CREDIT_RATIO_MAX,PREV_REFUSED_NEW_APP_CREDIT_RATIO_MEAN,PREV_REFUSED_NEW_APP_CREDIT_RATIO_MEDIAN,PREV_REFUSED_NEW_ANNUITY_CREDIT_RATIO_MIN,PREV_REFUSED_NEW_ANNUITY_CREDIT_RATIO_MAX,PREV_REFUSED_NEW_ANNUITY_CREDIT_RATIO_MEAN,PREV_REFUSED_NEW_ANNUITY_CREDIT_RATIO_MEDIAN,PREV_REFUSED_NEW_CREDIT_GOODS_RATIO_MIN,PREV_REFUSED_NEW_CREDIT_GOODS_RATIO_MAX,PREV_REFUSED_NEW_CREDIT_GOODS_RATIO_MEAN,PREV_REFUSED_NEW_CREDIT_GOODS_RATIO_MEDIAN,PREV_REFUSED_NEW_AMT_INTEREST_MIN,PREV_REFUSED_NEW_AMT_INTEREST_MAX,PREV_REFUSED_NEW_AMT_INTEREST_MEAN,PREV_REFUSED_NEW_AMT_INTEREST_MEDIAN,PREV_REFUSED_NEW_INTEREST_RATIO_MIN,PREV_REFUSED_NEW_INTEREST_RATIO_MAX,PREV_REFUSED_NEW_INTEREST_RATIO_MEAN,PREV_REFUSED_NEW_INTEREST_RATIO_MEDIAN,PREV_REFUSED_NEW_AMT_NEEDED_CREDIT_RATIO_MIN,PREV_REFUSED_NEW_AMT_NEEDED_CREDIT_RATIO_MAX,PREV_REFUSED_NEW_AMT_NEEDED_CREDIT_RATIO_MEAN,PREV_REFUSED_NEW_AMT_NEEDED_CREDIT_RATIO_MEDIAN,PREV_REFUSED_NEW_WEEKDAY_SIN_MIN,PREV_REFUSED_NEW_WEEKDAY_SIN_MAX,PREV_REFUSED_NEW_WEEKDAY_SIN_MEAN,PREV_REFUSED_NEW_WEEKDAY_SIN_MEDIAN,PREV_REFUSED_NEW_WEEKDAY_COS_MIN,PREV_REFUSED_NEW_WEEKDAY_COS_MAX,PREV_REFUSED_NEW_WEEKDAY_COS_MEAN,PREV_REFUSED_NEW_WEEKDAY_COS_MEDIAN
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,3951.000,3951.000,3951.000,3951.000,24835.5,24835.5,24835.50,24835.50,23787.0,23787.0,23787.00,23787.00,2520.0,2520.0,2520.0,2520.0,24835.5,24835.5,24835.5,24835.5,0.104326,0.104326,0.104326,0.104326,-1740,-1740,-1740.0,-1740.0,8.0,8.0,8.0,8.0,1.044079,1.044079,1.044079,1.044079,0.166099,0.166099,0.166099,0.166099,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100002,9251.775,9251.775,9251.775,9251.775,179055.0,179055.0,179055.00,179055.00,179055.0,179055.0,179055.00,179055.00,0.0,0.0,0.0,0.0,179055.0,179055.0,179055.0,179055.0,0.000000,0.000000,0.000000,0.000000,-606,-606,-606.0,-606.0,24.0,24.0,24.0,24.0,1.000000,1.000000,1.000000,1.000000,0.051670,0.051670,0.051670,0.051670,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100003,6737.310,98356.995,56553.990,64567.665,68809.5,900000.0,435436.50,337500.00,68053.5,1035882.0,484191.00,348637.50,0.0,6885.0,3442.5,3442.5,68809.5,900000.0,435436.5,337500.0,0.000000,0.100061,0.050030,0.050030,-2341,-746,-1305.0,-828.0,6.0,12.0,10.0,12.0,0.868825,1.011109,0.949329,0.968054,0.094950,0.185200,0.126383,0.099000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [135]:
final_prev_df.to_csv("/content/drive/MyDrive/p7/data/final_prev_df.csv", index=False)


## **POS_CASH_balance.csv**

This table contains the Monthly Balance Snapshots of previous Point of Sales and Cash Loans that the applicant had with Home Credit Group. 

In [137]:
pos = pd.read_csv('/content/drive/MyDrive/p7/data/POS_CASH_balance.csv',sep=',', encoding='utf-8', low_memory=False)

In [138]:
pos .head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [139]:
pos .shape

(10001358, 8)

In [140]:
pos .isnull().sum().sort_values(ascending= False).head(10)

CNT_INSTALMENT_FUTURE    26087
CNT_INSTALMENT           26071
SK_ID_PREV                   0
SK_ID_CURR                   0
MONTHS_BALANCE               0
NAME_CONTRACT_STATUS         0
SK_DPD                       0
SK_DPD_DEF                   0
dtype: int64

In [141]:
pos, cat_cols = one_hot_encoder(pos, nan_as_category=True)

In [142]:
# Features
aggregations = {
    'MONTHS_BALANCE': ['max', 'mean', 'size'],
    'SK_DPD': ['max', 'mean'],
    'SK_DPD_DEF': ['max', 'mean']
}
for cat in cat_cols:
    aggregations[cat] = ['mean']

pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])


In [143]:
# Count pos cash accounts
pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()

In [144]:
del pos
gc.collect()

342

In [145]:
pos_agg.head()

,POS_MONTHS_BALANCE_MAX,POS_MONTHS_BALANCE_MEAN,POS_MONTHS_BALANCE_SIZE,POS_SK_DPD_MAX,POS_SK_DPD_MEAN,POS_SK_DPD_DEF_MAX,POS_SK_DPD_DEF_MEAN,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Amortized debt_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Returned to the store_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_nan_MEAN,POS_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,
100001,-53,-72.555556,9,7,0.777778,7,0.777778,0.777778,0.0,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9
100002,-1,-10.000000,19,0,0.000000,0,0.000000,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,19
100003,-18,-43.785714,28,0,0.000000,0,0.000000,0.928571,0.0,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0,0.0,28
100004,-24,-25.500000,4,0,0.000000,0,0.000000,0.750000,0.0,0.0,0.0,0.250000,0.0,0.0,0.000000,0.0,0.0,4
100005,-15,-20.000000,11,0,0.000000,0,0.000000,0.818182,0.0,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11


In [146]:
pos_agg.to_csv("/content/drive/MyDrive/p7/data/pos_agg_final.csv", index=False)

## **installments_payments.csv**

This table contains the details about each installment of client's previous credits with Home Credit Group.

In [147]:
ins = pd.read_csv('/content/drive/MyDrive/p7/data/installments_payments.csv',sep=',', encoding='utf-8', low_memory=False)

In [148]:
ins .shape

(13605401, 8)

In [149]:
ins .head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [150]:
ins .isnull().sum().sort_values(ascending= False).head(5)

DAYS_ENTRY_PAYMENT        2905
AMT_PAYMENT               2905
SK_ID_PREV                   0
SK_ID_CURR                   0
NUM_INSTALMENT_VERSION       0
dtype: int64

In [151]:
ins, cat_cols = one_hot_encoder(ins, nan_as_category=True)

In [152]:
# Percentage and difference paid in each installment (amount paid and installment value)
ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']

# Days past due and days before due (no negative values)
ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)


In [153]:
# Features: Perform aggregations
aggregations = {
    'NUM_INSTALMENT_VERSION': ['nunique'],
    'DPD': ['max', 'mean', 'sum'],
    'DBD': ['max', 'mean', 'sum'],
    'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
    'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
    'AMT_INSTALMENT': ['max', 'mean', 'sum'],
    'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
    'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']}

cat_cols = [col for col in ins.columns if ins[col].dtypes == "O"]
for cat in cat_cols:
    aggregations[cat] = ['mean']
ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])

In [154]:
# Count installments accounts
ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()

In [155]:
del ins
gc.collect()

345

In [156]:
ins_agg.head()

,INSTAL_NUM_INSTALMENT_VERSION_NUNIQUE,INSTAL_DPD_MAX,INSTAL_DPD_MEAN,INSTAL_DPD_SUM,INSTAL_DBD_MAX,INSTAL_DBD_MEAN,INSTAL_DBD_SUM,INSTAL_PAYMENT_PERC_MAX,INSTAL_PAYMENT_PERC_MEAN,INSTAL_PAYMENT_PERC_SUM,INSTAL_PAYMENT_PERC_VAR,INSTAL_PAYMENT_DIFF_MAX,INSTAL_PAYMENT_DIFF_MEAN,INSTAL_PAYMENT_DIFF_SUM,INSTAL_PAYMENT_DIFF_VAR,INSTAL_AMT_INSTALMENT_MAX,INSTAL_AMT_INSTALMENT_MEAN,INSTAL_AMT_INSTALMENT_SUM,INSTAL_AMT_PAYMENT_MIN,INSTAL_AMT_PAYMENT_MAX,INSTAL_AMT_PAYMENT_MEAN,INSTAL_AMT_PAYMENT_SUM,INSTAL_DAYS_ENTRY_PAYMENT_MAX,INSTAL_DAYS_ENTRY_PAYMENT_MEAN,INSTAL_DAYS_ENTRY_PAYMENT_SUM,INSTAL_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,2,11.0,1.571429,11.0,36.0,8.857143,62.0,1.0,1.0,7.0,0.0,0.0,0.0,0.0,0.0,17397.900,5885.132143,41195.925,3951.000,17397.900,5885.132143,41195.925,-1628.0,-2195.000000,-15365.0,7
100002,2,0.0,0.000000,0.0,31.0,20.421053,388.0,1.0,1.0,19.0,0.0,0.0,0.0,0.0,0.0,53093.745,11559.247105,219625.695,9251.775,53093.745,11559.247105,219625.695,-49.0,-315.421053,-5993.0,19
100003,2,0.0,0.000000,0.0,14.0,7.160000,179.0,1.0,1.0,25.0,0.0,0.0,0.0,0.0,0.0,560835.360,64754.586000,1618864.650,6662.970,560835.360,64754.586000,1618864.650,-544.0,-1385.320000,-34633.0,25
100004,2,0.0,0.000000,0.0,11.0,7.666667,23.0,1.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,10573.965,7096.155000,21288.465,5357.250,10573.965,7096.155000,21288.465,-727.0,-761.666667,-2285.0,3
100005,2,1.0,0.111111,1.0,37.0,23.666667,213.0,1.0,1.0,9.0,0.0,0.0,0.0,0.0,0.0,17656.245,6240.205000,56161.845,4813.200,17656.245,6240.205000,56161.845,-470.0,-609.555556,-5486.0,9


In [157]:
ins_agg.to_csv("/content/drive/MyDrive/p7/data/ins_agg_final.csv", index=False)

## **credit_card_balance.csv**

This table consists of the monthly data related to any or multiple Credit Cards that the applicant had with the Home Credit Group. The table contains fields like balance, the credit limit, amount of drawings, etc. for each month of the credit card.

In [158]:
cc= pd.read_csv('/content/drive/MyDrive/p7/data/credit_card_balance.csv',sep=',', encoding='utf-8', low_memory=False)

In [159]:
cc.shape

(3840312, 23)

In [160]:
cc, cat_cols = one_hot_encoder(cc, nan_as_category=True)

In [161]:
# number of loans per customer
CCB = cc[0:]
grp = CCB.groupby(by = ['SK_ID_CURR'])['SK_ID_PREV'].nunique().reset_index().rename(index = str, columns = {'SK_ID_PREV': 'NO_LOANS'})
cc = CCB.merge(grp, on = ['SK_ID_CURR'], how = 'left')

# CNT_INSTALMENT_MATURE_CUM gives number of installments paid as of that month 
# The Max of this column shows the total number of installments made for that loan 

CCB = cc[0:]

# No of Loans per customer 

CCB = cc[0:]
grp = CCB.groupby(by = ['SK_ID_CURR'])['SK_ID_PREV'].nunique().reset_index().rename(index = str, columns = {'SK_ID_PREV': 'NO_LOANS'})
cc = CCB.merge(grp, on = ['SK_ID_CURR'], how = 'left')

In [162]:
del grp 
gc.collect()

131

In [163]:
    # No of Installments paid per Loan per Customer 

grp = CCB.groupby(by = ['SK_ID_CURR', 'SK_ID_PREV'])['CNT_INSTALMENT_MATURE_CUM'].max().reset_index().rename(index = str, columns = {'CNT_INSTALMENT_MATURE_CUM': 'NO_INSTALMENTS'})
grp1 = grp.groupby(by = ['SK_ID_CURR'])['NO_INSTALMENTS'].sum().reset_index().rename(index = str, columns = {'NO_INSTALMENTS': 'TOTAL_INSTALMENTS'})
cc = CCB.merge(grp1, on = ['SK_ID_CURR'], how = 'left')
del grp, grp1
gc.collect()

0

In [164]:
# Average Number of installments paid per loan 

cc['NEW_INSTALLMENTS_PER_LOAN'] = (cc['TOTAL_INSTALMENTS']/cc['NO_LOANS']).astype('uint32')
del cc['TOTAL_INSTALMENTS']
del cc['NO_LOANS']

# General aggregations
cc.drop(['SK_ID_PREV'], axis=1, inplace=True)
cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])

# Count credit card lines
cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()

del cc
gc.collect()

0

In [165]:
cc_agg.head()

,CC_MONTHS_BALANCE_MIN,CC_MONTHS_BALANCE_MAX,CC_MONTHS_BALANCE_MEAN,CC_MONTHS_BALANCE_SUM,CC_MONTHS_BALANCE_VAR,CC_AMT_BALANCE_MIN,CC_AMT_BALANCE_MAX,CC_AMT_BALANCE_MEAN,CC_AMT_BALANCE_SUM,CC_AMT_BALANCE_VAR,CC_AMT_CREDIT_LIMIT_ACTUAL_MIN,CC_AMT_CREDIT_LIMIT_ACTUAL_MAX,CC_AMT_CREDIT_LIMIT_ACTUAL_MEAN,CC_AMT_CREDIT_LIMIT_ACTUAL_SUM,CC_AMT_CREDIT_LIMIT_ACTUAL_VAR,CC_AMT_DRAWINGS_ATM_CURRENT_MIN,CC_AMT_DRAWINGS_ATM_CURRENT_MAX,CC_AMT_DRAWINGS_ATM_CURRENT_MEAN,CC_AMT_DRAWINGS_ATM_CURRENT_SUM,CC_AMT_DRAWINGS_ATM_CURRENT_VAR,CC_AMT_DRAWINGS_CURRENT_MIN,CC_AMT_DRAWINGS_CURRENT_MAX,CC_AMT_DRAWINGS_CURRENT_MEAN,CC_AMT_DRAWINGS_CURRENT_SUM,CC_AMT_DRAWINGS_CURRENT_VAR,CC_AMT_DRAWINGS_OTHER_CURRENT_MIN,CC_AMT_DRAWINGS_OTHER_CURRENT_MAX,CC_AMT_DRAWINGS_OTHER_CURRENT_MEAN,CC_AMT_DRAWINGS_OTHER_CURRENT_SUM,CC_AMT_DRAWINGS_OTHER_CURRENT_VAR,CC_AMT_DRAWINGS_POS_CURRENT_MIN,CC_AMT_DRAWINGS_POS_CURRENT_MAX,CC_AMT_DRAWINGS_POS_CURRENT_MEAN,CC_AMT_DRAWINGS_POS_CURRENT_SUM,CC_AMT_DRAWINGS_POS_CURRENT_VAR,CC_AMT_INST_MIN_REGULARITY_MIN,CC_AMT_INST_MIN_REGULARITY_MAX,CC_AMT_INST_MIN_REGULARITY_MEAN,CC_AMT_INST_MIN_REGULARITY_SUM,CC_AMT_INST_MIN_REGULARITY_VAR,...,CC_NAME_CONTRACT_STATUS_Approved_MAX,CC_NAME_CONTRACT_STATUS_Approved_MEAN,CC_NAME_CONTRACT_STATUS_Approved_SUM,CC_NAME_CONTRACT_STATUS_Approved_VAR,CC_NAME_CONTRACT_STATUS_Completed_MIN,CC_NAME_CONTRACT_STATUS_Completed_MAX,CC_NAME_CONTRACT_STATUS_Completed_MEAN,CC_NAME_CONTRACT_STATUS_Completed_SUM,CC_NAME_CONTRACT_STATUS_Completed_VAR,CC_NAME_CONTRACT_STATUS_Demand_MIN,CC_NAME_CONTRACT_STATUS_Demand_MAX,CC_NAME_CONTRACT_STATUS_Demand_MEAN,CC_NAME_CONTRACT_STATUS_Demand_SUM,CC_NAME_CONTRACT_STATUS_Demand_VAR,CC_NAME_CONTRACT_STATUS_Refused_MIN,CC_NAME_CONTRACT_STATUS_Refused_MAX,CC_NAME_CONTRACT_STATUS_Refused_MEAN,CC_NAME_CONTRACT_STATUS_Refused_SUM,CC_NAME_CONTRACT_STATUS_Refused_VAR,CC_NAME_CONTRACT_STATUS_Sent proposal_MIN,CC_NAME_CONTRACT_STATUS_Sent proposal_MAX,CC_NAME_CONTRACT_STATUS_Sent proposal_MEAN,CC_NAME_CONTRACT_STATUS_Sent proposal_SUM,CC_NAME_CONTRACT_STATUS_Sent proposal_VAR,CC_NAME_CONTRACT_STATUS_Signed_MIN,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_NEW_INSTALLMENTS_PER_LOAN_MIN,CC_NEW_INSTALLMENTS_PER_LOAN_MAX,CC_NEW_INSTALLMENTS_PER_LOAN_MEAN,CC_NEW_INSTALLMENTS_PER_LOAN_SUM,CC_NEW_INSTALLMENTS_PER_LOAN_VAR,CC_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100006,-6,-1,-3.5,-21,3.5,0.0,0.00,0.000000,0.000,0.000000e+00,270000,270000,270000.000000,1620000,0.000000e+00,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.000000,0.0,0.000000e+00,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.000000,0.000,0.000000e+00,...,0,0.0,0,0.0,0,0,0.000000,0,0.000000,0,0,0.0,0,0.0,0,0,0.0,0,0.0,0,0,0.0,0,0.0,0,0,0.0,0,0.0,0,0,0.0,0,0.0,0,0,0.0,0,0.0,6
100011,-75,-2,-38.5,-2849,462.5,0.0,189000.00,54482.111149,4031676.225,4.641321e+09,90000,180000,164189.189189,12150000,1.189060e+09,0.0,180000.0,2432.432432,180000.0,4.378378e+08,0.0,180000.0,2432.432432,180000.0,4.378378e+08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9000.0,3956.221849,288804.195,2.013991e+07,...,0,0.0,0,0.0,0,0,0.000000,0,0.000000,0,0,0.0,0,0.0,0,0,0.0,0,0.0,0,0,0.0,0,0.0,0,0,0.0,0,0.0,0,0,0.0,0,0.0,33,33,33.0,2442,0.0,74
100013,-96,-1,-48.5,-4656,776.0,0.0,161420.22,18159.919219,1743352.245,1.869473e+09,45000,157500,131718.750000,12645000,2.259252e+09,0.0,157500.0,6350.000000,571500.0,8.249688e+08,0.0,157500.0,5953.125000,571500.0,7.752530e+08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7875.0,1454.539551,129454.020,9.171263e+06,...,0,0.0,0,0.0,0,0,0.000000,0,0.000000,0,0,0.0,0,0.0,0,0,0.0,0,0.0,0,0,0.0,0,0.0,0,0,0.0,0,0.0,0,0,0.0,0,0.0,22,22,22.0,2112,0.0,96
100021,-18,-2,-10.0,-170,25.5,0.0,0.00,0.000000,0.000,0.000000e+00,6

In [166]:
cc_agg.to_csv("/content/drive/MyDrive/p7/data/cc_agg_final.csv", index=False)

## **Merging All Tables**


Now we will merge all the preprocessed tables with the application_train and application_test tables. The merges will be Left Outer Joins, such that all the current applications are preserved, as we have to model on them.

In [94]:
df_train_final= pd.read_csv('/content/drive/MyDrive/p7/data/df_train_final.csv')
bu_agg_final= pd.read_csv('/content/drive/MyDrive/p7/data/bu_agg_final.csv')

final_prev_df= pd.read_csv('/content/drive/MyDrive/p7/data/final_prev_df.csv')
pos_agg_final= pd.read_csv('/content/drive/MyDrive/p7/data/pos_agg_final.csv')
ins_agg_final= pd.read_csv('/content/drive/MyDrive/p7/data/ins_agg_final.csv')
cc_agg_final= pd.read_csv('/content/drive/MyDrive/p7/data/cc_agg_final.csv')

In [97]:
# Merging data
data = df_train_final.join(bu_agg_final, how='left', on='SK_ID_CURR')
del df_train_final, bu_agg_final

data = data.join(final_prev_df, how='left', on='SK_ID_CURR')
del final_prev_df

data = data.join(pos_agg_final, how='left', on='SK_ID_CURR')
del pos_agg_final

data = data.join(ins_agg_final, how='left', on='SK_ID_CURR')
del ins_agg_final

data = data.join(cc_agg_final, how='left', on='SK_ID_CURR')
del cc_agg_final

In [98]:
data.shape

(356249, 896)

In [99]:
data.to_csv("/content/drive/MyDrive/p7/data/NEW_data.csv", index=False)